In [55]:
import pandas as pd
import numpy as np

In [56]:
import json
import pandas as pd

# Load data from input.json
with open("input.json", "r") as file:
    data = json.load(file)

# Convert orders data to DataFrame
orders_df = pd.DataFrame(data["orders"])
orders_df.insert(0, "ID", range(1, len(orders_df) + 1))  # Add index starting from 1
orders_df.rename(columns={"delivery_x": "x", "delivery_y": "y", "deadline": "time", "package_weight": "weight"}, inplace=True)

# Convert drones data to DataFrame
drones_df = pd.DataFrame(data["drones"]["fleet"])
drones_df.insert(0, "Drone_ID", range(1, len(drones_df) + 1))  # Add index starting from 1
drones_df.rename(columns={"max_payload": "max_payload", "max_distance": "max_dist"}, inplace=True)

# Display DataFrames
orders_df



,ID,id,x,y,time,weight
0,1,O1,2,2,15,2
1,2,O2,3,3,30,8
2,3,O3,4,4,25,4
3,4,O4,13,12,40,8


In [57]:
drones_df

,Drone_ID,id,max_payload,max_dist,speed,available
0,1,D1,20,100,2.0,True
1,2,D2,25,40,1.5,True
2,3,D3,7,20,2.5,True


In [58]:

# orders = {
#     'ID':[1,2,3,4],
#     'x':[2,3,4,13],
#     'y':[2,3,4,12],
#     'time':[15,30,25,40],
#     'weight':[2,8,4,8]
# }

# drones = {
#     'Drone_ID':[1,2,3],
#     'max_payload' : [20,25,7],
#     'max_dist' : [100,40,20],
#     'speed' : [2,1.5,2.5],
#     'available' : ['true', 'true', 'true']
# }

drones_df = drones_df[drones_df['available']].reset_index(drop=True)

In [59]:
from itertools import permutations, combinations

order_ids = orders_df['ID'].tolist()

# Generate all combinations and their corresponding permutations
all_combos_perms = []
for r in range(1, len(order_ids) + 1):
    for combo in combinations(order_ids, r):
        all_combos_perms.extend(permutations(combo))

# Create dist_list based on order combos
dist_data = []
for combo in all_combos_perms:
    dist_list = [(orders_df.loc[orders_df['ID'] == order, 'x'].values[0], 
                  orders_df.loc[orders_df['ID'] == order, 'y'].values[0]) for order in combo]
    
    # Manhattan distance from (0,0) to each point
    distance_list = [abs(x) + abs(y) for (x, y) in dist_list]
    
    # Calculate relative distance between consecutive points
    relative_distance = [abs(dist_list[0][0]) + abs(dist_list[0][1])]  # First distance from origin
    for i in range(len(dist_list) - 1):
        x1, y1 = dist_list[i]
        x2, y2 = dist_list[i + 1]
        relative_distance.append(abs(x2 - x1) + abs(y2 - y1))  # Manhattan distance between points
    
    # Calculate cumulative distance (progressive sum of relative distances)
    cumulative_distance = [relative_distance[0]]  # First distance from origin
    for i in range(1, len(relative_distance)):
        cumulative_distance.append(cumulative_distance[-1] + relative_distance[i])  # Progressive sum
    
    # Calculate total distance (round trip from origin → all points → back to origin)
    total_distance = cumulative_distance[-1] + abs(dist_list[-1][0]) + abs(dist_list[-1][1])

    # Calculate total weight of the order combo
    total_weight = sum(orders_df.loc[orders_df['ID'].isin(combo), 'weight'])

    # Extract times of the orders in the combo
    time_list = orders_df.loc[orders_df['ID'].isin(combo), 'time'].tolist()

    # Drone-specific calculations
    drone_columns = {}
    for _, drone in drones_df.iterrows():
        drone_id = drone['Drone_ID']
        max_payload = drone['max_payload']
        max_dist = drone['max_dist']
        speed = drone['speed']

        # Check weight constraint
        weight_check = 1 if total_weight <= max_payload else 0
        
        # Check distance constraint
        distance_check = 1 if total_distance <= max_dist else 0
        
        # Check delivery time constraint
        delivery_time_check = 1 if all(time >= cum_dist / speed for time, cum_dist in zip(time_list, cumulative_distance)) else 0

        # Overall feasibility for this drone
        overall_check = 1 if (weight_check and distance_check and delivery_time_check) else 0

        # Add to dictionary
        drone_columns[f'drone_{drone_id}_weight'] = weight_check
        drone_columns[f'drone_{drone_id}_distance'] = distance_check
        drone_columns[f'drone_{drone_id}_delivery_time_constraints'] = delivery_time_check
        drone_columns[f'drone_{drone_id}_overall'] = overall_check

    # Append to data list
    dist_data.append({
        'Order_Combos': combo, 
        'dist_list': dist_list, 
        'distance_list': distance_list,
        'relative_distance': relative_distance,
        'cumulative_distance': cumulative_distance,
        'total_distance': total_distance,
        'weight': total_weight,
        'time': time_list,
        **drone_columns  # Unpack the dictionary for drone-specific columns
    })

# Create DataFrame
order_data_df = pd.DataFrame(dist_data)

# Display the DataFrame
order_data_df


,Order_Combos,dist_list,distance_list,relative_distance,cumulative_distance,total_distance,weight,time,drone_1_weight,drone_1_distance,drone_1_delivery_time_constraints,drone_1_overall,drone_2_weight,drone_2_distance,drone_2_delivery_time_constraints,drone_2_overall,drone_3_weight,drone_3_distance,drone_3_delivery_time_constraints,drone_3_overall
0,"(1,)","[(2, 2)]",[4],[4],[4],8,2,[15],1,1,1,1,1,1,1,1,1,1,1,1
1,"(2,)","[(3, 3)]",[6],[6],[6],12,8,[30],1,1,1,1,1,1,1,1,0,1,1,0
2,"(3,)","[(4, 4)]",[8],[8],[8],16,4,[25],1,1,1,1,1,1,1,1,1,1,1,1
3,"(4,)","[(13, 12)]",[25],[25],[25],50,8,[40],1,1,1,1,1,0,1,0,0,0,1,0
4,"(1, 2)","[(2, 2), (3, 3)]","[4, 6]","[4, 2]","[4, 6]",12,10,"[15, 30]",1,1,1,1,1,1,1,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,"(4, 1, 3, 2)","[(13, 12), (2, 2), (4, 4), (3, 3)]","[25, 4, 8, 6]","[25, 21, 4, 2]","[25, 46, 50, 52]",58,22,"[15, 30, 25, 40]",0,1,1,0,1,0,0,0,0,0,1,0
60,"(4, 2, 1, 3)","[(13, 12), (3, 3), (2, 2), (4, 4)]","[25, 6, 4, 8]","[25, 19, 2, 4]","[25, 44, 46, 50]",58,22,"[15, 30, 25, 40]",0,1,1,0,1,0,0,0,0,0,1,0
61,"(4, 2, 3, 1)","[(13, 12), (3, 3), (4, 4), (2, 2)]","[25, 6, 8, 4]","[25, 19, 2, 4]","[25, 44, 46, 50]",54,22,"[15, 30, 25, 40]",0,1,1,0,1,0,0,0,0,0,1,0
62,"(4, 3, 1, 2)","[(13, 12), (4, 4), (2, 2), (3, 3)]","[25, 8, 4, 6]","[25, 17, 4, 2]","[25, 42, 46, 48]",54,22,"[15, 30, 25, 40]",0,1,1,0,1,0,0,0,0,0,1,0


In [60]:
import pandas as pd
from itertools import product

combos_lists = []
for drone in drones_df['Drone_ID']:
    valid_combos = list(order_data_df[order_data_df[f'drone_{drone}_overall'] == 1]['Order_Combos'])
    combos_lists.append(valid_combos + [0])  # Add 0 as a placeholder for "no assignment"

# Generate all possible drone assignments
all_combinations = list(product(*combos_lists))

# Function to check if any order appears more than once in a combination
def is_valid_combination(combination):
    orders = set()
    for combo in combination:
        if combo == 0:  # Ignore placeholder
            continue
        for order in combo:
            if order in orders:
                return False  # Order is repeated
            orders.add(order)
    return True

# Filter out invalid combinations
valid_combinations = [combo for combo in all_combinations if is_valid_combination(combo)]

valid_combinations  # This contains only unique order assignments across drones


[((1,), (2,), (3,)),
 ((1,), (2,), 0),
 ((1,), (3,), 0),
 ((1,), (2, 3), 0),
 ((1,), (3, 2), 0),
 ((1,), 0, (3,)),
 ((1,), 0, 0),
 ((2,), (1,), (3,)),
 ((2,), (1,), 0),
 ((2,), (3,), (1,)),
 ((2,), (3,), 0),
 ((2,), (1, 3), 0),
 ((2,), (3, 1), 0),
 ((2,), 0, (1,)),
 ((2,), 0, (3,)),
 ((2,), 0, (1, 3)),
 ((2,), 0, (3, 1)),
 ((2,), 0, 0),
 ((3,), (1,), 0),
 ((3,), (2,), (1,)),
 ((3,), (2,), 0),
 ((3,), (1, 2), 0),
 ((3,), (2, 1), 0),
 ((3,), 0, (1,)),
 ((3,), 0, 0),
 ((4,), (1,), (3,)),
 ((4,), (1,), 0),
 ((4,), (2,), (1,)),
 ((4,), (2,), (3,)),
 ((4,), (2,), (1, 3)),
 ((4,), (2,), (3, 1)),
 ((4,), (2,), 0),
 ((4,), (3,), (1,)),
 ((4,), (3,), 0),
 ((4,), (1, 2), (3,)),
 ((4,), (1, 2), 0),
 ((4,), (2, 1), (3,)),
 ((4,), (2, 1), 0),
 ((4,), (1, 3), 0),
 ((4,), (3, 1), 0),
 ((4,), (2, 3), (1,)),
 ((4,), (2, 3), 0),
 ((4,), (3, 2), (1,)),
 ((4,), (3, 2), 0),
 ((4,), (1, 2, 3), 0),
 ((4,), (1, 3, 2), 0),
 ((4,), (2, 1, 3), 0),
 ((4,), (2, 3, 1), 0),
 ((4,), (3, 1, 2), 0),
 ((4,), (3, 2, 1), 0

In [61]:
# Create DataFrame with valid combinations
valid_combos_df = pd.DataFrame(valid_combinations, columns=[f'drone_{drone}' for drone in drones_df['Drone_ID']])

# Add a column for the full valid combination
valid_combos_df.insert(0, 'Valid_combinations', valid_combos_df.apply(tuple, axis=1))

# Display the result
valid_combos_df


,Valid_combinations,drone_1,drone_2,drone_3
0,"((1,), (2,), (3,))","(1,)","(2,)","(3,)"
1,"((1,), (2,), 0)","(1,)","(2,)",0
2,"((1,), (3,), 0)","(1,)","(3,)",0
3,"((1,), (2, 3), 0)","(1,)","(2, 3)",0
4,"((1,), (3, 2), 0)","(1,)","(3, 2)",0
...,...,...,...,...
200,"(0, 0, (1,))",0,0,"(1,)"
201,"(0, 0, (3,))",0,0,"(3,)"
202,"(0, 0, (1, 3))",0,0,"(1, 3)"
203,"(0, 0, (3, 1))",0,0,"(3, 1)"


In [62]:
# Create DataFrame with valid combinations
valid_combos_df = pd.DataFrame(valid_combinations, columns=[f'drone_{drone}' for drone in drones_df['Drone_ID']])

# Add a column for the full valid combination
valid_combos_df.insert(0, 'Valid_combinations', valid_combos_df.apply(tuple, axis=1))

# Function to retrieve required columns for each drone
def get_drone_metrics(combo, column_name):
    if combo == 0:
        return 0  # If no assignment, set value to 0
    return order_data_df.loc[order_data_df['Order_Combos'] == combo, column_name].values[0]

# Iterate over each drone and extract required columns
for drone in drones_df['Drone_ID']:
    valid_combos_df[f'drone_{drone}_weight'] = valid_combos_df[f'drone_{drone}'].apply(lambda x: get_drone_metrics(x, 'weight'))
    valid_combos_df[f'drone_{drone}_order_coord'] = valid_combos_df[f'drone_{drone}'].apply(lambda x: get_drone_metrics(x, 'dist_list'))
    valid_combos_df[f'drone_{drone}_distance'] = valid_combos_df[f'drone_{drone}'].apply(lambda x: get_drone_metrics(x, 'distance_list'))
    valid_combos_df[f'drone_{drone}_relative_distance'] = valid_combos_df[f'drone_{drone}'].apply(lambda x: get_drone_metrics(x, 'relative_distance'))
    valid_combos_df[f'drone_{drone}_cumulative_distance'] = valid_combos_df[f'drone_{drone}'].apply(lambda x: get_drone_metrics(x, 'cumulative_distance'))
    valid_combos_df[f'drone_{drone}_total_distance'] = valid_combos_df[f'drone_{drone}'].apply(lambda x: get_drone_metrics(x, 'total_distance'))

# Display updated DataFrame
valid_combos_df.tail(10)



,Valid_combinations,drone_1,drone_2,drone_3,drone_1_weight,drone_1_order_coord,drone_1_distance,drone_1_relative_distance,drone_1_cumulative_distance,drone_1_total_distance,...,drone_2_distance,drone_2_relative_distance,drone_2_cumulative_distance,drone_2_total_distance,drone_3_weight,drone_3_order_coord,drone_3_distance,drone_3_relative_distance,drone_3_cumulative_distance,drone_3_total_distance
195,"(0, (1, 3, 2), 0)",0,"(1, 3, 2)",0,0,0,0,0,0,0,...,"[4, 8, 6]","[4, 4, 2]","[4, 8, 10]",16,0,0,0,0,0,0
196,"(0, (2, 1, 3), 0)",0,"(2, 1, 3)",0,0,0,0,0,0,0,...,"[6, 4, 8]","[6, 2, 4]","[6, 8, 12]",20,0,0,0,0,0,0
197,"(0, (2, 3, 1), 0)",0,"(2, 3, 1)",0,0,0,0,0,0,0,...,"[6, 8, 4]","[6, 2, 4]","[6, 8, 12]",16,0,0,0,0,0,0
198,"(0, (3, 1, 2), 0)",0,"(3, 1, 2)",0,0,0,0,0,0,0,...,"[8, 4, 6]","[8, 4, 2]","[8, 12, 14]",20,0,0,0,0,0,0
199,"(0, (3, 2, 1), 0)",0,"(3, 2, 1)",0,0,0,0,0,0,0,...,"[8, 6, 4]","[8, 2, 2]","[8, 10, 12]",16,0,0,0,0,0,0
200,"(0, 0, (1,))",0,0,"(1,)",0,0,0,0,0,0,...,0,0,0,0,2,"[(2, 2)]",[4],[4],[4],8
201,"(0, 0, (3,))",0,0,"(3,)",0,0,0,0,0,0,...,0,0,0,0,4,"[(4, 4)]",[8],[8],[8],16
202,"(0, 0, (1, 3))",0,0,"(1, 3)",0,0,0,0,0,0,...,0,0,0,0,6,"[(2, 2), (4, 4)]","[4, 8]","[4, 4]","[4, 8]",16
203,"(0, 0, (3, 1))",0,0,"(3, 1)",0,0,0,0,0,0,...,0,0,0,0,6,"[(4, 4), (2, 2)]","[8, 4]","[8, 4]","[8, 12]",16
204,"(0, 0, 0)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
# Function to get the last value of a list column
def get_last_value(value):
    return value[-1] if isinstance(value, list) and value else 0

# Iterate over each drone and compute total time
for drone, speed in zip(drones_df['Drone_ID'], drones_df['speed']):
    valid_combos_df[f'drone_{drone}_total_time'] = (
        valid_combos_df[f'drone_{drone}_cumulative_distance'].apply(get_last_value) / speed
    )

# Display the updated DataFrame
valid_combos_df


,Valid_combinations,drone_1,drone_2,drone_3,drone_1_weight,drone_1_order_coord,drone_1_distance,drone_1_relative_distance,drone_1_cumulative_distance,drone_1_total_distance,...,drone_2_total_distance,drone_3_weight,drone_3_order_coord,drone_3_distance,drone_3_relative_distance,drone_3_cumulative_distance,drone_3_total_distance,drone_1_total_time,drone_2_total_time,drone_3_total_time
0,"((1,), (2,), (3,))","(1,)","(2,)","(3,)",2,"[(2, 2)]",[4],[4],[4],8,...,12,4,"[(4, 4)]",[8],[8],[8],16,2.0,4.000000,3.2
1,"((1,), (2,), 0)","(1,)","(2,)",0,2,"[(2, 2)]",[4],[4],[4],8,...,12,0,0,0,0,0,0,2.0,4.000000,0.0
2,"((1,), (3,), 0)","(1,)","(3,)",0,2,"[(2, 2)]",[4],[4],[4],8,...,16,0,0,0,0,0,0,2.0,5.333333,0.0
3,"((1,), (2, 3), 0)","(1,)","(2, 3)",0,2,"[(2, 2)]",[4],[4],[4],8,...,16,0,0,0,0,0,0,2.0,5.333333,0.0
4,"((1,), (3, 2), 0)","(1,)","(3, 2)",0,2,"[(2, 2)]",[4],[4],[4],8,...,16,0,0,0,0,0,0,2.0,6.666667,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,"(0, 0, (1,))",0,0,"(1,)",0,0,0,0,0,0,...,0,2,"[(2, 2)]",[4],[4],[4],8,0.0,0.000000,1.6
201,"(0, 0, (3,))",0,0,"(3,)",0,0,0,0,0,0,...,0,4,"[(4, 4)]",[8],[8],[8],16,0.0,0.000000,3.2
202,"(0, 0, (1, 3))",0,0,"(1, 3)",0,0,0,0,0,0,...,0,6,"[(2, 2), (4, 4)]","[4, 8]","[4, 4]","[4, 8]",16,0.0,0.000000,3.2
203,"(0, 0, (3, 1))",0,0,"(3, 1)",0,0,0,0,0,0,...,0,6,"[(4, 4), (2, 2)]","[8, 4]","[8, 4]","[8, 12]",16,0.0,0.000000,4.8


In [64]:
# Summing up total time and total distance across all drones
valid_combos_df["Total_Time"] = valid_combos_df[[f'drone_{drone}_total_time' for drone in drones_df["Drone_ID"]]].sum(axis=1)
valid_combos_df["Total_Distance"] = valid_combos_df[[f'drone_{drone}_total_distance' for drone in drones_df["Drone_ID"]]].sum(axis=1)

# Display the updated DataFrame
valid_combos_df


,Valid_combinations,drone_1,drone_2,drone_3,drone_1_weight,drone_1_order_coord,drone_1_distance,drone_1_relative_distance,drone_1_cumulative_distance,drone_1_total_distance,...,drone_3_order_coord,drone_3_distance,drone_3_relative_distance,drone_3_cumulative_distance,drone_3_total_distance,drone_1_total_time,drone_2_total_time,drone_3_total_time,Total_Time,Total_Distance
0,"((1,), (2,), (3,))","(1,)","(2,)","(3,)",2,"[(2, 2)]",[4],[4],[4],8,...,"[(4, 4)]",[8],[8],[8],16,2.0,4.000000,3.2,9.200000,36
1,"((1,), (2,), 0)","(1,)","(2,)",0,2,"[(2, 2)]",[4],[4],[4],8,...,0,0,0,0,0,2.0,4.000000,0.0,6.000000,20
2,"((1,), (3,), 0)","(1,)","(3,)",0,2,"[(2, 2)]",[4],[4],[4],8,...,0,0,0,0,0,2.0,5.333333,0.0,7.333333,24
3,"((1,), (2, 3), 0)","(1,)","(2, 3)",0,2,"[(2, 2)]",[4],[4],[4],8,...,0,0,0,0,0,2.0,5.333333,0.0,7.333333,24
4,"((1,), (3, 2), 0)","(1,)","(3, 2)",0,2,"[(2, 2)]",[4],[4],[4],8,...,0,0,0,0,0,2.0,6.666667,0.0,8.666667,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,"(0, 0, (1,))",0,0,"(1,)",0,0,0,0,0,0,...,"[(2, 2)]",[4],[4],[4],8,0.0,0.000000,1.6,1.600000,8
201,"(0, 0, (3,))",0,0,"(3,)",0,0,0,0,0,0,...,"[(4, 4)]",[8],[8],[8],16,0.0,0.000000,3.2,3.200000,16
202,"(0, 0, (1, 3))",0,0,"(1, 3)",0,0,0,0,0,0,...,"[(2, 2), (4, 4)]","[4, 8]","[4, 4]","[4, 8]",16,0.0,0.000000,3.2,3.200000,16
203,"(0, 0, (3, 1))",0,0,"(3, 1)",0,0,0,0,0,0,...,"[(4, 4), (2, 2)]","[8, 4]","[8, 4]","[8, 12]",16,0.0,0.000000,4.8,4.800000,16


In [65]:
# Get all unique order IDs
all_order_ids = set(orders_df["ID"])

# Function to check if all orders are present in a combination
def check_all_orders_present(combination):
    combo_orders = set()
    for drone_combo in combination:
        if drone_combo != 0:  # Ignore empty assignments (0)
            combo_orders.update(drone_combo)
    return 1 if combo_orders == all_order_ids else 0

# Apply function to create the new column
valid_combos_df["contains_all_orders"] = valid_combos_df["Valid_combinations"].apply(check_all_orders_present)

# Display the updated DataFrame
valid_combos_df


,Valid_combinations,drone_1,drone_2,drone_3,drone_1_weight,drone_1_order_coord,drone_1_distance,drone_1_relative_distance,drone_1_cumulative_distance,drone_1_total_distance,...,drone_3_distance,drone_3_relative_distance,drone_3_cumulative_distance,drone_3_total_distance,drone_1_total_time,drone_2_total_time,drone_3_total_time,Total_Time,Total_Distance,contains_all_orders
0,"((1,), (2,), (3,))","(1,)","(2,)","(3,)",2,"[(2, 2)]",[4],[4],[4],8,...,[8],[8],[8],16,2.0,4.000000,3.2,9.200000,36,0
1,"((1,), (2,), 0)","(1,)","(2,)",0,2,"[(2, 2)]",[4],[4],[4],8,...,0,0,0,0,2.0,4.000000,0.0,6.000000,20,0
2,"((1,), (3,), 0)","(1,)","(3,)",0,2,"[(2, 2)]",[4],[4],[4],8,...,0,0,0,0,2.0,5.333333,0.0,7.333333,24,0
3,"((1,), (2, 3), 0)","(1,)","(2, 3)",0,2,"[(2, 2)]",[4],[4],[4],8,...,0,0,0,0,2.0,5.333333,0.0,7.333333,24,0
4,"((1,), (3, 2), 0)","(1,)","(3, 2)",0,2,"[(2, 2)]",[4],[4],[4],8,...,0,0,0,0,2.0,6.666667,0.0,8.666667,24,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,"(0, 0, (1,))",0,0,"(1,)",0,0,0,0,0,0,...,[4],[4],[4],8,0.0,0.000000,1.6,1.600000,8,0
201,"(0, 0, (3,))",0,0,"(3,)",0,0,0,0,0,0,...,[8],[8],[8],16,0.0,0.000000,3.2,3.200000,16,0
202,"(0, 0, (1, 3))",0,0,"(1, 3)",0,0,0,0,0,0,...,"[4, 8]","[4, 4]","[4, 8]",16,0.0,0.000000,3.2,3.200000,16,0
203,"(0, 0, (3, 1))",0,0,"(3, 1)",0,0,0,0,0,0,...,"[8, 4]","[8, 4]","[8, 12]",16,0.0,0.000000,4.8,4.800000,16,0


In [66]:
# Function to count the number of orders in a combination
def count_orders(combination):
    combo_orders = set()
    for drone_combo in combination:
        if drone_combo != 0:  # Ignore empty assignments (0)
            combo_orders.update(drone_combo)
    return len(combo_orders)

# Apply function to create the new column
valid_combos_df["count"] = valid_combos_df["Valid_combinations"].apply(count_orders)

# Display the updated DataFrame
valid_combos_df


,Valid_combinations,drone_1,drone_2,drone_3,drone_1_weight,drone_1_order_coord,drone_1_distance,drone_1_relative_distance,drone_1_cumulative_distance,drone_1_total_distance,...,drone_3_relative_distance,drone_3_cumulative_distance,drone_3_total_distance,drone_1_total_time,drone_2_total_time,drone_3_total_time,Total_Time,Total_Distance,contains_all_orders,count
0,"((1,), (2,), (3,))","(1,)","(2,)","(3,)",2,"[(2, 2)]",[4],[4],[4],8,...,[8],[8],16,2.0,4.000000,3.2,9.200000,36,0,3
1,"((1,), (2,), 0)","(1,)","(2,)",0,2,"[(2, 2)]",[4],[4],[4],8,...,0,0,0,2.0,4.000000,0.0,6.000000,20,0,2
2,"((1,), (3,), 0)","(1,)","(3,)",0,2,"[(2, 2)]",[4],[4],[4],8,...,0,0,0,2.0,5.333333,0.0,7.333333,24,0,2
3,"((1,), (2, 3), 0)","(1,)","(2, 3)",0,2,"[(2, 2)]",[4],[4],[4],8,...,0,0,0,2.0,5.333333,0.0,7.333333,24,0,3
4,"((1,), (3, 2), 0)","(1,)","(3, 2)",0,2,"[(2, 2)]",[4],[4],[4],8,...,0,0,0,2.0,6.666667,0.0,8.666667,24,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,"(0, 0, (1,))",0,0,"(1,)",0,0,0,0,0,0,...,[4],[4],8,0.0,0.000000,1.6,1.600000,8,0,1
201,"(0, 0, (3,))",0,0,"(3,)",0,0,0,0,0,0,...,[8],[8],16,0.0,0.000000,3.2,3.200000,16,0,1
202,"(0, 0, (1, 3))",0,0,"(1, 3)",0,0,0,0,0,0,...,"[4, 4]","[4, 8]",16,0.0,0.000000,3.2,3.200000,16,0,2
203,"(0, 0, (3, 1))",0,0,"(3, 1)",0,0,0,0,0,0,...,"[8, 4]","[8, 12]",16,0.0,0.000000,4.8,4.800000,16,0,2


In [67]:
# Sort the DataFrame by Total_Time in ascending order
valid_combos_df = valid_combos_df.sort_values(by="Total_Time", ascending=True).reset_index(drop=True)

In [68]:
# Case 1: Select the row where contains_all_orders == 1 and has the least Total_Time
if (valid_combos_df["contains_all_orders"] == 1).any():
    final_output = valid_combos_df[valid_combos_df["contains_all_orders"] == 1].nsmallest(1, "Total_Time")
else:
    # Case 2: Find the max count and then select the row with the least Total_Time
    max_count = valid_combos_df["count"].max()
    final_output = valid_combos_df[valid_combos_df["count"] == max_count].nsmallest(1, "Total_Time")

# Display the final output
final_output


,Valid_combinations,drone_1,drone_2,drone_3,drone_1_weight,drone_1_order_coord,drone_1_distance,drone_1_relative_distance,drone_1_cumulative_distance,drone_1_total_distance,...,drone_3_relative_distance,drone_3_cumulative_distance,drone_3_total_distance,drone_1_total_time,drone_2_total_time,drone_3_total_time,Total_Time,Total_Distance,contains_all_orders,count
86,"((2, 3, 4), 0, (1,))","(2, 3, 4)",0,"(1,)",20,"[(3, 3), (4, 4), (13, 12)]","[6, 8, 25]","[6, 2, 17]","[6, 8, 25]",50,...,[4],[4],8,12.5,0.0,1.6,14.1,58,1,4


In [69]:
drone_list = []
order_list = []
distance_list = []
for drone,id in zip(list(drones_df['Drone_ID']), list(drones_df['id'])):
    orders = list(final_output[f'drone_{drone}'])[0]
    orders_1 = []
    if orders == 0:
         orders_1 = []
    else:
        for order in orders:
            orders_1.append(list(orders_df[orders_df['ID'] == order]['id'])[0])
    order_list.append(orders_1)
    distance_list.append(list(final_output[f'drone_{drone}_total_distance'])[0])
    drone_list.append(id)

json_dict = {"assignments":
             [{"drone":d,
               "orders": o,
               "total_distance": l} for d,o,l in zip(drone_list, order_list, distance_list)]
             }

# Save to a JSON file
with open("output.json", "w") as file:
    json.dump(json_dict, file, indent=4)